In [788]:
from bs4 import BeautifulSoup
import pandas as pd
import os
import re
import requests
import logging
import threading

<h1 style="color:yellow;"> Scraping our data</h1>
<h3> First thing we need to do to start the project would be to gather our data</h3>
<p> We would like to find a reliable website of a very known fighting organization , this way we can trust the data since it is an official fighting organization </p>
<p>We are going to scrape ESPN official website for getting our data </p>
<a href="http://www.espn.com/mma/fighters">  Link to ESPN's website where we can observe the fighter list </a>

In [789]:
user_agent = {'User-agent': 'Mozilla/5.0'}

<h3> First we would like to have a list with links for all pages of of the fither list </h3>
<h3> first page will contain fithers with name started with letter A</h3>
<h3> first page will contain fithers with name started with letter Z</h3>

In [790]:
url1="http://www.espn.com/mma/fighters"
response1 = requests.get(url1,headers=user_agent)
pagesAToZ=[]
# we would like to add the first page (the page that contains fighters with letter A)
pagesAToZ.append(url1)
soup1 = BeautifulSoup(response1.content, "html.parser")
div_t=soup1.find("div",attrs={"class":"mod-content"})
hred_t=div_t.findAll("a")
for t in hred_t :
    pagesAToZ.append(url1+t['href'])


<h3>Let's print the urls so we can make sure everything looks fine </h3>

In [791]:
print(pagesAToZ)

['http://www.espn.com/mma/fighters', 'http://www.espn.com/mma/fighters?search=b', 'http://www.espn.com/mma/fighters?search=c', 'http://www.espn.com/mma/fighters?search=d', 'http://www.espn.com/mma/fighters?search=e', 'http://www.espn.com/mma/fighters?search=f', 'http://www.espn.com/mma/fighters?search=g', 'http://www.espn.com/mma/fighters?search=h', 'http://www.espn.com/mma/fighters?search=i', 'http://www.espn.com/mma/fighters?search=j', 'http://www.espn.com/mma/fighters?search=k', 'http://www.espn.com/mma/fighters?search=l', 'http://www.espn.com/mma/fighters?search=m', 'http://www.espn.com/mma/fighters?search=n', 'http://www.espn.com/mma/fighters?search=o', 'http://www.espn.com/mma/fighters?search=p', 'http://www.espn.com/mma/fighters?search=q', 'http://www.espn.com/mma/fighters?search=r', 'http://www.espn.com/mma/fighters?search=s', 'http://www.espn.com/mma/fighters?search=t', 'http://www.espn.com/mma/fighters?search=u', 'http://www.espn.com/mma/fighters?search=v', 'http://www.espn.c

<h3> Another function we would like to writw is a functions that scrapes the most inner page that contain previews fights stats , we would like to get a path as an argument</h3>

In [792]:
def scrapeinnerPageForFightStats(path):
    
    wins =""
    loses =""
    decision = ""
    winByKo = ""
    loseByKo= ""
    winbySubmission=""
    loseBySubmission=""
    page_link='http://www.espn.com'+path
   #  print("pagelink is" , page_link )
    response= requests.get(page_link)
    soup = BeautifulSoup(response.content)
    regex_winsLosesDecision = "W-L-D(\d)-(\d)-(\d)"
    regex_ko="\(T\)KO(\d)-(\d)"
    regex_sub="SUB(\d)-(\d)"
    ul_t = soup.find("ul", attrs={"class":"StatBlock__Content flex list ph4 pv3 justify-between"}) 
    if(ul_t==None):
       print("No unordered list atrr")

    if( (not hasattr(ul_t,'findAll')) ):
      return wins , loses , decision , winByKo , loseByKo, winbySubmission , loseBySubmission

   
    if( not ul_t.findAll("li") ):
      return wins , loses , decision , winByKo , loseByKo, winbySubmission , loseBySubmission
        
    li_list=ul_t.findAll("li")
    for li in li_list:
       text=li.text
    #    print(text)
       winLoseDecStats_regex_res=re.search(regex_winsLosesDecision , text)
       koStats_regex_res=re.search(regex_ko , text)
       subStats_regex_res=re.search(regex_sub , text)
       if(winLoseDecStats_regex_res):
           wins=winLoseDecStats_regex_res.group(1)
           loses=winLoseDecStats_regex_res.group(2)
           decision=winLoseDecStats_regex_res.group(3)
        #    print(winLoseDecStats_regex_res)
       elif(koStats_regex_res):
           winByKo=koStats_regex_res.group(1)
           loseByKo=koStats_regex_res.group(2)
        #    print(koStats_regex_res)
       elif(subStats_regex_res):
           winbySubmission=subStats_regex_res.group(1)
           loseBySubmission=subStats_regex_res.group(2)
        #    print(subStats_regex_res)
    logging.info(wins,[wins , loses , decision , winByKo , loseByKo, winbySubmission , loseBySubmission])
    print(wins , loses , decision , winByKo , loseByKo, winbySubmission , loseBySubmission)
    return wins , loses , decision , winByKo , loseByKo, winbySubmission , loseBySubmission


<h3> Let's make sure we get the right return params and that the regexes are fine</h3>
<p> we will expect three matches and output params ('0', '1', '0', '0', '0', '0', '1') </p>

In [793]:
# scrapeinnerPageForFightStats('/mma/fighter/_/id/3043549/niina-aaltonen')

<h3> Cool , We have a list of links to each page we want to scrape and the stats from the inner pages! </h3>
<h4>Now we would like to build a function for scraping each page  </h4>

<p>
/The structure of the page containes a long table with two columns . <br>
First Column - Name <br>
Sec Column - Country <br>

the Name is clickable and redirect us to another page with more details of the fighter
 </p>

In [794]:
def getTableRows(page_link):
    response= requests.get(page_link)
    soup = BeautifulSoup(response.content)
    table = soup.find(lambda tag: tag.name=='table') 
    rows = table.findAll(lambda tag: tag.name=='tr')
    # we would like to remove the first two rows bc it doesnt contain any data
    del rows[0]
    del rows[0]
    return rows

In [795]:
def runOnSingleDataRowGetNameCountryBio(row):
    #currently runs only on the first row
        full_name , link_to_bio , country = "" , "" ,""
        cells = row.findAll(lambda tag: tag.name=='td')
        if(cells == None or (not cells)):
            return 
        full_name = cells[0].find("a").text
        full_name=full_name.split(", ")
        first_name=full_name[0]
        last_name=full_name[1]
        link_to_bio = cells[0].find("a")["href"]
        country=cells[1].text
        full_name=first_name+" "+last_name 
        return full_name , link_to_bio ,country



In [796]:
def scrapeTheWholePage(page_url):
    data=[]
    rows=getTableRows(page_url)
    for row in rows:
        full_name  , bio_link , country = runOnSingleDataRowGetNameCountryBio(row)
        wins , loses , decision , winByKo , loseByKo, winbySubmission , loseBySubmission =scrapeinnerPageForFightStats(bio_link)
        # print(full_name , bio_link , country)
        # print(wins , loses , decision , winByKo , loseByKo, winbySubmission , loseBySubmission)
        listOfParams=[full_name  , country , wins , loses , decision , winByKo , loseByKo, winbySubmission , loseBySubmission ]
        # print(listOfParams)
        print(full_name)
        logging.info(full_name)
        data.append(listOfParams)
        # now we need to return DF with the whole page data
        #next were gonna gather all the dfs together and save it to a file
    #return df
    
    df = pd.DataFrame(data, columns=['Full_Name', 'Country', 'Wins' , 'Loses' ,'Decision' , 'Wins_By_KO' , 'Loses_By_KO', 'Wins_By_Submission' , 'Loses_By_Submission' ])
    return df

    

<h3> test ourselves on the main page </h3>

In [797]:
# scrapeTheWholePage('http://www.espn.com/mma/fighters')

<h2>Now we need to orginize our data in a csv file  , since we have lots of data , we will split it to a few files</h2>

In [798]:
def dataframToCsvFile(df , file_name, path):
    os.makedirs(path, exist_ok=True) 
    df.to_csv(path+file_name+'.csv' , index=False)  

In [799]:
def scrapePageAndSaveToCsv(page_link , page_num):
        df=scrapeTheWholePage(page_link)
        dataframToCsvFile(df,'dataPage'+str(page_num))
        


<h1> We're going to use threads to scrape all pages in pharllel </h1>

In [800]:

def scrapeMultiPagesAnd(pagesToScrap):
    threadList=[]
    for i in range(len(pagesToScrap)):
             threadObj=threading.Thread(target=scrapePageAndSaveToCsv, args=(pagesToScrap[i],i+6,))
             threadObj.start();
             threadList.append(threadObj)
    for t in threadList:
             t.join();

In [801]:
pagesToScrap=pagesAToZ
# scrapeMultiPagesAnd(pagesToScrap)

<h3> Perfect ! now we have all our data scraped and reay for the next step 🥳 🥳 </h3>


<h1 > Next Step <span style="color:green;"> Cleaning our Data 🧽 🧹<span> </h1>

In [802]:
def load_csv(filename):
    return pd.read_csv(filename)

In [803]:
df=load_csv('./scrapedData/dataPage25.csv')
df.head(10)


,Full_Name,Country,Wins,Loses,Decision,Wins_By_KO,Loses_By_KO,Wins_By_Submission,Loses_By_Submission
0,Zaba Arkadiusz,NaN,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,Zaba Marcin,NaN,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,Zabolotny Dmitry,NaN,1.0,3.0,0.0,0.0,2.0,1.0,1.0
3,Zabrocki Michael,NaN,0.0,1.0,0.0,0.0,1.0,0.0,0.0
4,Zac Charlie,NaN,0.0,2.0,0.0,0.0,0.0,0.0,2.0
5,Zachrich Luke,USA,NaN,NaN,NaN,4.0,1.0,7.0,3.0
6,Zaczkiewicz Sylwia,POL,0.0,1.0,0.0,0.0,1.0,0.0,0.0
7,Zadernovsky Igor,POL,0.0,2.0,0.0,0.0,0.0,0.0,2.0
8,Zadruzynski Filip,POL,0.0,1.0,0.0,0.0,0.0,0.0,1.0
9,Zafir Anton,AUS,7.0,3.0,0.0,3.0,3.0,3.0,0.0


In [804]:
def fillNansWihtValues(df , col_names_list , value_to_fill_with):
    df_cpy=df.copy();
    for col_name in col_names_list:
         df_cpy[col_name]=df_cpy[col_name].fillna(value_to_fill_with).copy()
    return df_cpy


In [805]:
def removeColFromDf(df , cols_list_to_remove):
    df_cpy=df.copy()
    return df_cpy.drop(cols_list_to_remove , axis=1)

In [806]:
def addColToDf(df , col_name , col_fixed_value):
    df_cpy=df.copy()
    df_cpy[col_name]=col_fixed_value
    return df_cpy


In [807]:

def checkNameAndChangeChampionVal(df , list_of_champions):
    # for each row  -> if name in list_of_champions ->isChamp= 1
    # df = df.reset_index()
    for index,row in df.iterrows():
        if row['Full_Name'] in list_of_champions :
            row['Champion']=1
            print("  Found Champion  ")
            print(" Index = " , index)
            print("Name = " , row['Full_Name'])
            print("changes Champion Val from 0 to ",row['Champion'] )
        

In [808]:
def remove_rows_with_NA_vals_in_specific_col(df , specific_col ):
    df_cpy=df.copy()
    return df_cpy.dropna(subset=[specific_col])


In [809]:
def remove_duplicates(df, col):
    df_cpy=df.copy()
    df_cpy=df_cpy.drop_duplicates(subset=[col])
    return df_cpy

In [810]:
def dataCleanUpFlow(out_folder_loc , list_of_file_names):
    for file in list_of_file_names:
        scrapedData_filename='./scrapedData/'+file+'.csv'

        df=load_csv(scrapedData_filename)
        df=remove_duplicates(df , 'Full_Name')
        df=remove_rows_with_NA_vals_in_specific_col(df , 'Country' )
        df=remove_rows_with_NA_vals_in_specific_col(df , 'Full_Name' )
        df=fillNansWihtValues(df , ['Decision' , 'Wins_By_KO' , 'Loses_By_KO' , 'Wins_By_Submission','Loses_By_Submission'] , 0)
        df=removeColFromDf(df , ['Wins' , 'Loses'])
        df=addColToDf(df , 'Champion' , 0)
        dataframToCsvFile(df,file+'.csv' , out_folder_loc)

        
        
        

In [811]:
# checkNameAndChangeChampionVal(df , list_of_champions)

In [812]:
listOfFiles=['dataPage'+str(i)  for i in range(26)]
path='./cleanedData/'
dataCleanUpFlow(path,listOfFiles)